In [1]:
import pandas as pd
import glob as glob

## Importing Statcast Data

Here we do some initial data treatment.

In [2]:
# Import Statcast Data

for year in range(2008,2022):
    df = pd.read_csv(f'Statcast Raw Data/{year}.csv')
    df = df.rename({' first_name': 'first_name'}, axis = 1)
    df['first_name'] = df['first_name'].str.lstrip()
    df['full_name'] = df[['first_name','last_name']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    df = df[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
    df['full_name'] = df['full_name'].replace({' Jr.':'', '-':' ', ' III':''},regex=True)     
               
    df.to_csv(f'Initial Treated Data/{year}_initial.csv')

Dropping some of the columns with Nan values. We won't drop the player_id column because we will use that as our index.
We will fill the blanks with the correct player ID.

### Players' ID source #1 - Statcast Database

The source we'll use to create a primary ID database is the Statcast data. From 2015 to 2021, all players have their respective ID listed.

In [3]:
path = 'Initial Treated Data'
all_files = glob.glob(path + "/*initial.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

# Extracting players IDs to create a primary database

p_id_db = frame[['full_name', 'player_id']]
p_id_db = p_id_db[p_id_db['player_id'].notna()]

In [4]:
frame = frame.drop(columns = 'Unnamed: 0')

Considering we might have two different players with the same name and without id, let's delete potential duplicates based on stats

In [5]:
stats_cols = ['full_name',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']

frame = frame.drop_duplicates(subset=stats_cols, keep='first')

### Dataframe only with players without ID

In [6]:
with_id = frame[frame['player_id'].notna()]
with_id

last_name first_name         full_name  player_id  year  player_age  \
0           Colon    Bartolo     Bartolo Colon   112526.0  2017          44   
1          Beltre     Adrian     Adrian Beltre   134181.0  2017          38   
2         Beltran     Carlos    Carlos Beltran   136860.0  2017          40   
3           Werth     Jayson      Jayson Werth   150029.0  2017          38   
4          Arroyo    Bronson    Bronson Arroyo   276520.0  2017          40   
...           ...        ...               ...        ...   ...         ...   
9090        Lopez      Nicky       Nicky Lopez   670032.0  2019          24   
9091      Alvarez     Yordan    Yordan Alvarez   670541.0  2019          22   
9092     Brosseau       Mike     Mike Brosseau   670712.0  2019          25   
9093     Richards     Trevor   Trevor Richards   670950.0  2019          26   
9094  Margevicius       Nick  Nick Margevicius   676606.0  2019          23   

      b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  b_home_run  \
0       19          20             0         0  ...         0           0   
1      340         389           106        66  ...         1          17   
2      467         509           108        65  ...         0          14   
3      252         289            57        36  ...         1          10   
4       26          29             4         4  ...         0           0   
...    ...         ...           ...       ...  ...       ...         ...   
9090   379         402            91        65  ...         2           2   
9091   313         369            98        45  ...         0          27   
9092   132         142            36        23  ...         0           6   
9093    30          34             3         3  ...         0           0   
9094    15          17             3         3  ...         0           0   

      b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0              10       0         50.0           0.0        0.000   
1              52      39         13.4          10.0        0.312   
2             102      33         20.0           6.5        0.231   
3              69      35         23.9          12.1        0.226   
4              10       0         34.5           0.0        0.154   
...           ...     ...          ...           ...          ...   
9090           51      18         12.7           4.5        0.240   
9091           94      52         25.5          14.1        0.313   
9092           39       7         27.5           4.9        0.273   
9093           18       2         52.9           5.9        0.100   
9094            7       0         41.2           0.0        0.200   

      slg_percent  on_base_percent  on_base_plus_slg  
0           0.000            0.000             0.000  
1           0.532            0.383             0.915  
2           0.383            0.283             0.666  
3           0.393            0.322             0.715  
4           0.154            0.138             0.292  
...           ...              ...               ...  
9090        0.325            0.274             0.598  
9091        0.655            0.412             1.067  
9092        0.462            0.317             0.779  
9093        0.100            0.147             0.247  
9094        0.200            0.176             0.376  

[4793 rows x 21 columns]

### Dataframe only with players without ID

In [7]:
no_id = frame[frame['player_id'].isna()]
no_id

last_name       first_name                  full_name  player_id  year  \
692        Cruz     Luis Alfonso          Luis Alfonso Cruz        NaN  2012   
693   Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
694      Werner           Andrew              Andrew Werner        NaN  2012   
695      Profar        Jurickson           Jurickson Profar        NaN  2012   
696     Calhoun             Kole               Kole Calhoun        NaN  2012   
...         ...              ...                        ...        ...   ...   
9806     Blanco            Henry               Henry Blanco        NaN  2010   
9807   Bautista             Jose              Jose Bautista        NaN  2010   
9808     Ausmus             Brad                Brad Ausmus        NaN  2010   
9809   Anderson           Garret            Garret Anderson        NaN  2010   
9810      Abreu            Bobby                Bobby Abreu        NaN  2010   

      player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
692           28   283         296            84        58  ...         0   
693           22    14          16             3         2  ...         0   
694           25    12          14             1         1  ...         0   
695           19    17          17             3         0  ...         0   
696           25    23          25             4         3  ...         0   
...          ...   ...         ...           ...       ...  ...       ...   
9806          39   130         144            28        21  ...         0   
9807          30   569         683           148        56  ...         3   
9808          41    63          71            14        12  ...         0   
9809          38   155         163            28        19  ...         1   
9810          36   573         667           146        84  ...         1   

      b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
692            6           34       9         11.5           3.0        0.297   
693            0            2       2         12.5          12.5        0.214   
694            0            4       0         28.6           0.0        0.083   
695            1            4       0         23.5           0.0        0.176   
696            0            6       2         24.0           8.0        0.174   
...          ...          ...     ...          ...           ...          ...   
9806           2           26      11         18.1           7.6        0.215   
9807          54          116     100         17.0          14.6        0.260   
9808           0           15       7         21.1           9.9        0.222   
9809           2           34       5         20.9           3.1        0.181   
9810          20          132      87         19.8          13.0        0.255   

      slg_percent  on_base_percent  on_base_plus_slg  
692         0.431            0.321             0.752  
693         0.286            0.313             0.598  
694         0.083            0.071             0.155  
695         0.471            0.176             0.647  
696         0.217            0.240             0.457  
...           ...              ...               ...  
9806        0.300            0.271             0.571  
9807        0.617            0.378             0.995  
9808        0.254            0.310             0.564  
9809        0.271            0.202             0.473  
9810        0.435            0.352             0.787  

[5018 rows x 21 columns]

### Players' ID source #2 - WAR Database

Our WAR database contains data from players since the 1960's, including all players IDs.

In [8]:
from unidecode import unidecode

bat_data_war = pd.read_csv('war_daily_bat.txt', sep=",", encoding = 'utf8')
bat_data_war = bat_data_war[bat_data_war['year_ID']>=2008]

In [9]:
bat_data_war = bat_data_war.rename({'mlb_ID': 'player_id', 'name_common': 'full_name', 'year_ID': 'year', 'age':'player_age','PA':'b_total_pa','lg_ID':'league'}, axis = 1)

player_id_bat_war = bat_data_war[['full_name','player_id', 'year','b_total_pa']]
player_id_bat_war['full_name'] = player_id_bat_war['full_name'].apply(unidecode)
player_id_bat_war.reset_index(drop = True)
player_id_bat_war = player_id_bat_war.drop_duplicates()
player_id_bat_war.sort_values(by='full_name')

original_names = bat_data_war[['player_id','full_name']].drop_duplicates()
original_names['full_name'] = original_names['full_name'].replace({'AJ Pollock': 'A.J. Pollock'}, regex = True)
original_names.to_csv('original_names.csv')

/var/folders/d8/89h35qn507zfzwn9__35p_bc0000gn/T/ipykernel_68308/3496699432.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_id_bat_war['full_name'] = player_id_bat_war['full_name'].apply(unidecode)


After some tries to get the most of the WAR database, we noticed that some values in the 'full_name' column were affecting the code effectiveness. With this in mind, the changes below were added:

In [10]:
player_id_bat_war['full_name'] = player_id_bat_war['full_name'].replace({
    'AJ Pollock':'A.J. Pollock', 
    'Chin-lung Hu':'Chin-Lung Hu',
    'Henderson Alvarez III':'Henderson Alvarez',
    'Hung-Chih Kuo':'Hong-Chih Kuo',
    'Hyun Jin Ryu':'Hyun-Jin Ryu',
    'JB Shuck':'J.B. Shuck',
    'Luis Cruz':'Luis Alfonso Cruz',
    'Luis Rodriguez':'Luis Antonio Rodriguez',
    'Mike Ryan':'Michael Ryan',
    'Mike Difelice':'Mike DiFelice',
    'Osvaldo Martinez': 'Ozzie Martinez',
    'Rick van den Hurk': 'Rick Van Den Hurk',
    'Russ Mitchell':'Russell Mitchell',
    'Val Pascucci':'Valentino Pascucci',
    'Vidal Nuno III': 'Vidal Nuno',
    'Wil Ledezma':'Wilfredo Ledezma',
    ' Jr.':'',
    ' III':'',
    '-':' ' 
}, regex = True)
player_id_bat_war['full_name'] = player_id_bat_war['full_name'].str.rstrip()

## Combining the datasets

#### First filling the Nan player ids with data from the Statcast dataset

Here we get three versions of the players' id database from the WAR data. As you can see, each version of the database has less details. This might help us to address data with different details.

In [11]:
player_id_bat_war

full_name  player_id  year  b_total_pa
3       David Aardsma   430911.0  2008         1.0
4       David Aardsma   430911.0  2009         0.0
5       David Aardsma   430911.0  2010         0.0
6       David Aardsma   430911.0  2012         0.0
7       David Aardsma   430911.0  2013         0.0
...               ...        ...   ...         ...
118754    Mike Zunino   572287.0  2020        84.0
118755    Mike Zunino   572287.0  2021       375.0
118787      Tony Zych   543964.0  2015         0.0
118788      Tony Zych   543964.0  2016         0.0
118789      Tony Zych   543964.0  2017         0.0

[19722 rows x 4 columns]

In [12]:
player_id_bat_war_2 = player_id_bat_war.drop(columns = ['b_total_pa'])
player_id_bat_war_2

full_name  player_id  year
3       David Aardsma   430911.0  2008
4       David Aardsma   430911.0  2009
5       David Aardsma   430911.0  2010
6       David Aardsma   430911.0  2012
7       David Aardsma   430911.0  2013
...               ...        ...   ...
118754    Mike Zunino   572287.0  2020
118755    Mike Zunino   572287.0  2021
118787      Tony Zych   543964.0  2015
118788      Tony Zych   543964.0  2016
118789      Tony Zych   543964.0  2017

[19722 rows x 3 columns]

First we merge the data using the most variables as binding points as we possibly can.

In [13]:
df2t = pd.merge(no_id, player_id_bat_war, how = 'left', on=['full_name','year','b_total_pa'])
df2t = df2t.drop(columns = 'player_id_x')
df2t = df2t.rename({'player_id_y':'player_id'},axis=1)
df2t = df2t[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
df2t

last_name       first_name                  full_name  player_id  year  \
0          Cruz     Luis Alfonso          Luis Alfonso Cruz   458501.0  2012   
1     Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
2        Werner           Andrew              Andrew Werner   599819.0  2012   
3        Profar        Jurickson           Jurickson Profar   595777.0  2012   
4       Calhoun             Kole               Kole Calhoun   594777.0  2012   
...         ...              ...                        ...        ...   ...   
5013     Blanco            Henry               Henry Blanco   111072.0  2010   
5014   Bautista             Jose              Jose Bautista   430832.0  2010   
5015     Ausmus             Brad                Brad Ausmus   110385.0  2010   
5016   Anderson           Garret            Garret Anderson   110236.0  2010   
5017      Abreu            Bobby                Bobby Abreu   110029.0  2010   

      player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0             28   283         296            84        58  ...         0   
1             22    14          16             3         2  ...         0   
2             25    12          14             1         1  ...         0   
3             19    17          17             3         0  ...         0   
4             25    23          25             4         3  ...         0   
...          ...   ...         ...           ...       ...  ...       ...   
5013          39   130         144            28        21  ...         0   
5014          30   569         683           148        56  ...         3   
5015          41    63          71            14        12  ...         0   
5016          38   155         163            28        19  ...         1   
5017          36   573         667           146        84  ...         1   

      b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0              6           34       9         11.5           3.0        0.297   
1              0            2       2         12.5          12.5        0.214   
2              0            4       0         28.6           0.0        0.083   
3              1            4       0         23.5           0.0        0.176   
4              0            6       2         24.0           8.0        0.174   
...          ...          ...     ...          ...           ...          ...   
5013           2           26      11         18.1           7.6        0.215   
5014          54          116     100         17.0          14.6        0.260   
5015           0           15       7         21.1           9.9        0.222   
5016           2           34       5         20.9           3.1        0.181   
5017          20          132      87         19.8          13.0        0.255   

      slg_percent  on_base_percent  on_base_plus_slg  
0           0.431            0.321             0.752  
1           0.286            0.313             0.598  
2           0.083            0.071             0.155  
3           0.471            0.176             0.647  
4           0.217            0.240             0.457  
...           ...              ...               ...  
5013        0.300            0.271             0.571  
5014        0.617            0.378             0.995  
5015        0.254            0.310             0.564  
5016        0.271            0.202             0.473  
5017        0.435            0.352             0.787  

[5018 rows x 21 columns]

Let's separate the rows with and without player id after this iteration

In [14]:
df2t_with_id = df2t[df2t['player_id'].notna()]

In [15]:
df2t_without_id = df2t[df2t['player_id'].isna()]
df2t_without_id

last_name       first_name                  full_name  player_id  year  \
1     Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
55      Eovaldi           Nathan             Nathan Eovaldi        NaN  2012   
78       Thames             Eric                Eric Thames        NaN  2012   
97      Komatsu             Erik               Erik Komatsu        NaN  2012   
120      Segura             Jean                Jean Segura        NaN  2012   
...         ...              ...                        ...        ...   ...   
4986        Lee           Derrek                 Derrek Lee        NaN  2010   
4997    Guillen             Jose               Jose Guillen        NaN  2010   
5000   Gonzalez             Alex              Alex Gonzalez        NaN  2010   
5002    Edmonds              Jim                Jim Edmonds        NaN  2010   
5007     Castro             Juan                Juan Castro        NaN  2010   

      player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
1             22    14          16             3         2  ...         0   
55            22    32          36             3         3  ...         0   
78            26   271         290            63        39  ...         3   
97            25    51          58            11        11  ...         0   
120           22   151         166            39        32  ...         3   
...          ...   ...         ...           ...       ...  ...       ...   
4986          35   547         626           142        88  ...         0   
4997          34   524         577           135        92  ...         2   
5000          33   595         640           149        81  ...         3   
5002          40   246         272            68        34  ...         0   
5007          38   129         140            25        20  ...         0   

      b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
1              0            2       2         12.5          12.5        0.214   
55             0           20       1         55.6           2.8        0.094   
78             9           87      15         30.0           5.2        0.232   
97             0            5       6          8.6          10.3        0.216   
120            0           23      13         13.9           7.8        0.258   
...          ...          ...     ...          ...           ...          ...   
4986          19          134      73         21.4          11.7        0.260   
4997          19          113      32         19.6           5.5        0.258   
5000          23          118      31         18.4           4.8        0.250   
5002          11           60      24         22.1           8.8        0.276   
5007           0           25       8         17.9           5.7        0.194   

      slg_percent  on_base_percent  on_base_plus_slg  
1           0.286            0.313             0.598  
55          0.094            0.111             0.205  
78          0.399            0.272             0.671  
97          0.216            0.293             0.509  
120         0.325            0.313             0.638  
...           ...              ...               ...  
4986        0.428            0.347             0.774  
4997        0.416            0.314             0.730  
5000        0.447            0.292             0.739  
5002        0.504            0.342             0.846  
5007        0.233            0.236             0.468  

[454 rows x 21 columns]

Now let's try getting some more player ids with the less restrictive dataset

In [16]:
df3t = pd.merge(df2t_without_id, player_id_bat_war_2, how = 'left', on=['full_name','year'])
df3t = df3t.drop_duplicates()
df3t = df3t.drop(columns=['player_id_x'],axis=1)
df3t = df3t.rename({'player_id_y':'player_id'},axis=1)
df3t = df3t[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
df3t

last_name       first_name                  full_name  player_id  year  \
0    Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
1      Eovaldi           Nathan             Nathan Eovaldi   543135.0  2012   
3       Thames             Eric                Eric Thames   519346.0  2012   
5      Komatsu             Erik               Erik Komatsu   518900.0  2012   
7       Segura             Jean                Jean Segura   516416.0  2012   
..         ...              ...                        ...        ...   ...   
879        Lee           Derrek                 Derrek Lee   117601.0  2010   
881    Guillen             Jose               Jose Guillen   115229.0  2010   
883   Gonzalez             Alex              Alex Gonzalez   136460.0  2010   
885    Edmonds              Jim                Jim Edmonds   113744.0  2010   
887     Castro             Juan                Juan Castro   112128.0  2010   

     player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0            22    14          16             3         2  ...         0   
1            22    32          36             3         3  ...         0   
3            26   271         290            63        39  ...         3   
5            25    51          58            11        11  ...         0   
7            22   151         166            39        32  ...         3   
..          ...   ...         ...           ...       ...  ...       ...   
879          35   547         626           142        88  ...         0   
881          34   524         577           135        92  ...         2   
883          33   595         640           149        81  ...         3   
885          40   246         272            68        34  ...         0   
887          38   129         140            25        20  ...         0   

     b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0             0            2       2         12.5          12.5        0.214   
1             0           20       1         55.6           2.8        0.094   
3             9           87      15         30.0           5.2        0.232   
5             0            5       6          8.6          10.3        0.216   
7             0           23      13         13.9           7.8        0.258   
..          ...          ...     ...          ...           ...          ...   
879          19          134      73         21.4          11.7        0.260   
881          19          113      32         19.6           5.5        0.258   
883          23          118      31         18.4           4.8        0.250   
885          11           60      24         22.1           8.8        0.276   
887           0           25       8         17.9           5.7        0.194   

     slg_percent  on_base_percent  on_base_plus_slg  
0          0.286            0.313             0.598  
1          0.094            0.111             0.205  
3          0.399            0.272             0.671  
5          0.216            0.293             0.509  
7          0.325            0.313             0.638  
..           ...              ...               ...  
879        0.428            0.347             0.774  
881        0.416            0.314             0.730  
883        0.447            0.292             0.739  
885        0.504            0.342             0.846  
887        0.233            0.236             0.468  

[455 rows x 21 columns]

In [17]:
df3t_with_id = df3t[df3t['player_id'].notna()]

In [18]:
df3t_without_id = df3t[df3t['player_id'].isna()]
df3t_without_id

last_name       first_name                  full_name  player_id  year  \
0     Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
15         Aoki        Norichika             Norichika Aoki        NaN  2012   
20        Niese         Jonathon             Jonathon Niese        NaN  2012   
21     Tolleson            Steve             Steve Tolleson        NaN  2012   
69        Morse          Michael              Michael Morse        NaN  2012   
134       Shuck               JB                   JB Shuck        NaN  2011   
143       Niese         Jonathon             Jonathon Niese        NaN  2011   
144       Field            Tommy                Tommy Field        NaN  2011   
169       Morse          Michael              Michael Morse        NaN  2011   
260       Morse          Michael              Michael Morse        NaN  2008   
344       Lopez            Raffy                Raffy Lopez        NaN  2014   
349         Ryu         Hyun-Jin               Hyun Jin Ryu        NaN  2014   
356  den Dekker          Matthew         Matthew den Dekker        NaN  2014   
383        Aoki        Norichika             Norichika Aoki        NaN  2014   
384       Niese         Jonathon             Jonathon Niese        NaN  2014   
385    Tolleson            Steve             Steve Tolleson        NaN  2014   
390       Perez      Juan Carlos          Juan Carlos Perez        NaN  2014   
451       Morse          Michael              Michael Morse        NaN  2014   
485  den Dekker          Matthew         Matthew den Dekker        NaN  2013   
486         Ryu         Hyun Jin               Hyun Jin Ryu        NaN  2013   
489     Pollock               AJ                 AJ Pollock        NaN  2013   
492       Shuck               JB                   JB Shuck        NaN  2013   
499   Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2013   
508        Aoki        Norichika             Norichika Aoki        NaN  2013   
518       Niese         Jonathon             Jonathon Niese        NaN  2013   
521       Field            Tommy                Tommy Field        NaN  2013   
526       Perez      Juan Carlos          Juan Carlos Perez        NaN  2013   
567       Morse          Michael              Michael Morse        NaN  2013   
646       Morse          Michael              Michael Morse        NaN  2009   
755       Niese         Jonathon             Jonathon Niese        NaN  2010   
760    Tolleson            Steve             Steve Tolleson        NaN  2010   
802       Morse          Michael              Michael Morse        NaN  2010   

     player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0            22    14          16             3         2  ...         0   
15           30   520         588           150        99  ...         4   
20           26    55          68            12        12  ...         0   
21           29    71          76            13         8  ...         0   
69           30   406         430           118        82  ...         1   
134          24    81          92            22        19  ...         1   
143          25    49          60             3         2  ...         1   
144          24    48          51            13        13  ...         0   
169          29   522         575           158        91  ...         0   
260          26     9          11             2         1  ...         0   
344          27    11          14             2         2  ...         0   
349          27    47          56             7         5  ...         0   
356          27   152         174            38        27  ...         0   
383          32   491         549           140       111  ...         6   
384          28    55          65             5         4  ...         0   
385          31   170         189            43        31  ...         2   
390          28   100         109            17         9  ...         0 

Now, for the remaining 32 datapoints without ID, let's apply the player ID data from the Statcast dataframe. At this point, the problem to correlate the dataset and the players' ID from the WAR table is how the names are written. 

In [19]:
df4t = pd.merge(df3t_without_id, p_id_db, how = 'left', on=['full_name'])
df4t = df4t.drop_duplicates()
df4t = df4t.drop(columns=['player_id_x'],axis=1)
df4t = df4t.rename({'player_id_y':'player_id'},axis=1)
df4t = df4t[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
df4t

last_name       first_name                  full_name  player_id  year  \
0    Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
1         Aoki        Norichika             Norichika Aoki   493114.0  2012   
4        Niese         Jonathon             Jonathon Niese   477003.0  2012   
6     Tolleson            Steve             Steve Tolleson   476270.0  2012   
7        Morse          Michael              Michael Morse   434604.0  2012   
9        Shuck               JB                   JB Shuck   543776.0  2011   
13       Niese         Jonathon             Jonathon Niese   477003.0  2011   
15       Field            Tommy                Tommy Field   474494.0  2011   
16       Morse          Michael              Michael Morse   434604.0  2011   
18       Morse          Michael              Michael Morse   434604.0  2008   
20       Lopez            Raffy                Raffy Lopez   607257.0  2014   
22         Ryu         Hyun-Jin               Hyun Jin Ryu   547943.0  2014   
25  den Dekker          Matthew         Matthew den Dekker   544925.0  2014   
28        Aoki        Norichika             Norichika Aoki   493114.0  2014   
31       Niese         Jonathon             Jonathon Niese   477003.0  2014   
33    Tolleson            Steve             Steve Tolleson   476270.0  2014   
34       Perez      Juan Carlos          Juan Carlos Perez   543633.0  2014   
35       Morse          Michael              Michael Morse   434604.0  2014   
37  den Dekker          Matthew         Matthew den Dekker   544925.0  2013   
40         Ryu         Hyun Jin               Hyun Jin Ryu   547943.0  2013   
43     Pollock               AJ                 AJ Pollock   572041.0  2013   
49       Shuck               JB                   JB Shuck   543776.0  2013   
53   Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2013   
54        Aoki        Norichika             Norichika Aoki   493114.0  2013   
57       Niese         Jonathon             Jonathon Niese   477003.0  2013   
59       Field            Tommy                Tommy Field   474494.0  2013   
60       Perez      Juan Carlos          Juan Carlos Perez   543633.0  2013   
61       Morse          Michael              Michael Morse   434604.0  2013   
63       Morse          Michael              Michael Morse   434604.0  2009   
65       Niese         Jonathon             Jonathon Niese   477003.0  2010   
67    Tolleson            Steve             Steve Tolleson   476270.0  2010   
68       Morse          Michael              Michael Morse   434604.0  2010   

    player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0           22    14          16             3         2  ...         0   
1           30   520         588           150        99  ...         4   
4           26    55          68            12        12  ...         0   
6           29    71          76            13         8  ...         0   
7           30   406         430           118        82  ...         1   
9           24    81          92            22        19  ...         1   
13          25    49          60             3         2  ...         1   
15          24    48          51            13        13  ...         0   
16          29   522         575           158        91  ...         0   
18          26     9          11             2         1  ...         0   
20          27    11          14             2         2  ...         0   
22          27    47          56             7         5  ...         0   
25          27   152         174            38        27  ...         0   
28          32   491         549           140       111  ...         6   
31          28    55          65             5         4  ...         0   
33          31   170         189            43        31  ...         2   
34          28   100         109            17         9  ...         0   
35          32   438         482           122 

In [20]:
df4t_with_id = df4t[df4t['player_id'].notna()]

In [21]:
df4t_without_id = df4t[df4t['player_id'].isna()]
df4t_without_id

last_name       first_name                  full_name  player_id  year  \
0   Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2012   
53  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez        NaN  2013   

    player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0           22    14          16             3         2  ...         0   
53          23     9          10             1         1  ...         0   

    b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0            0            2       2         12.5          12.5        0.214   
53           0            4       1         40.0          10.0        0.111   

    slg_percent  on_base_percent  on_base_plus_slg  
0         0.286            0.313             0.598  
53        0.111            0.200             0.311  

[2 rows x 21 columns]

Now we can see we only got one player left!

Manually checking the databases, we observe that there are two players named Henry Rodriguez that played in the MLB between 2008 and 2021. Searching online the player's ID from the WAR database, we discovered that the ID 514719 corresponds to our missing player! (https://www.thebaseballcube.com/content/player/152792/). Therefore, let's add this player's ID manually!

In [22]:
player_id_bat_war.loc[player_id_bat_war['player_id'] == 514719, 'full_name'] = 'Henry Alejandro Rodriguez'

In [23]:
df5t = pd.merge(df4t_without_id, player_id_bat_war, how = 'left', on=['full_name','year','b_total_pa'])
df5t = df5t.drop_duplicates()
df5t = df5t.drop(columns=['player_id_x'],axis=1)
df5t = df5t.rename({'player_id_y':'player_id'},axis=1)
df5t = df5t[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]
df5t

last_name       first_name                  full_name  player_id  year  \
0  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez   514719.0  2012   
1  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez   514719.0  2013   

   player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0          22    14          16             3         2  ...         0   
1          23     9          10             1         1  ...         0   

   b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0           0            2       2         12.5          12.5        0.214   
1           0            4       1         40.0          10.0        0.111   

   slg_percent  on_base_percent  on_base_plus_slg  
0        0.286            0.313             0.598  
1        0.111            0.200             0.311  

[2 rows x 21 columns]

In [24]:
df5t_with_id = df5t[df5t['player_id'].notna()]

In [25]:
df5t_without_id = df5t[df5t['player_id'].isna()]
df5t_without_id

Empty DataFrame
Columns: [last_name, first_name, full_name, player_id, year, player_age, b_ab, b_total_pa, b_total_hits, b_single, b_double, b_triple, b_home_run, b_strikeout, b_walk, b_k_percent, b_bb_percent, batting_avg, slg_percent, on_base_percent, on_base_plus_slg]
Index: []

[0 rows x 21 columns]

Great! Now all our players have their respective IDs!

#### Concatenate all the total dataframe pieces

In [26]:
pieces = [with_id, df2t_with_id, df3t_with_id, df3t_with_id, df5t_with_id]

complete_data = pd.concat(pieces).reset_index(drop=True)
complete_data

last_name       first_name                  full_name  player_id  year  \
0          Colon          Bartolo              Bartolo Colon   112526.0  2017   
1         Beltre           Adrian              Adrian Beltre   134181.0  2017   
2        Beltran           Carlos             Carlos Beltran   136860.0  2017   
3          Werth           Jayson               Jayson Werth   150029.0  2017   
4         Arroyo          Bronson             Bronson Arroyo   276520.0  2017   
...          ...              ...                        ...        ...   ...   
10200   Gonzalez             Alex              Alex Gonzalez   136460.0  2010   
10201    Edmonds              Jim                Jim Edmonds   113744.0  2010   
10202     Castro             Juan                Juan Castro   112128.0  2010   
10203  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez   514719.0  2012   
10204  Rodriguez  Henry Alejandro  Henry Alejandro Rodriguez   514719.0  2013   

       player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0              44    19          20             0         0  ...         0   
1              38   340         389           106        66  ...         1   
2              40   467         509           108        65  ...         0   
3              38   252         289            57        36  ...         1   
4              40    26          29             4         4  ...         0   
...           ...   ...         ...           ...       ...  ...       ...   
10200          33   595         640           149        81  ...         3   
10201          40   246         272            68        34  ...         0   
10202          38   129         140            25        20  ...         0   
10203          22    14          16             3         2  ...         0   
10204          23     9          10             1         1  ...         0   

       b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  \
0               0           10       0         50.0           0.0   
1              17           52      39         13.4          10.0   
2              14          102      33         20.0           6.5   
3              10           69      35         23.9          12.1   
4               0           10       0         34.5           0.0   
...           ...          ...     ...          ...           ...   
10200          23          118      31         18.4           4.8   
10201          11           60      24         22.1           8.8   
10202           0           25       8         17.9           5.7   
10203           0            2       2         12.5          12.5   
10204           0            4       1         40.0          10.0   

       batting_avg  slg_percent  on_base_percent  on_base_plus_slg  
0            0.000        0.000            0.000             0.000  
1            0.312        0.532            0.383             0.915  
2            0.231        0.383            0.283             0.666  
3            0.226        0.393            0.322             0.715  
4            0.154        0.154            0.138             0.292  
...            ...          ...              ...               ...  
10200        0.250        0.447            0.292             0.739  
10201        0.276        0.504            0.342             0.846  
10202        0.194        0.233            0.236             0.468  
10203        0.214        0.286            0.313             0.598  
10204        0.111        0.111            0.200             0.311  

[10205 rows x 21 columns]

#### Comparing the previous full dataframe that did not have all the ids with the new complete dataframe

In [27]:
print(frame.shape)
print(complete_data.shape)

(9811, 21)
(10205, 21)


We can see that the the number of rows is different, and the number or columns is ok. We need to find why there's a difference in the amount of rows. Let's check the names.

#### Retrieve Original Names

To avoid naming errors (for example same players named differently) we will convert the player's name in the 'with_id' table to the name on the original names file

In [28]:
complete_data_2 = pd.merge(complete_data, original_names, how = 'left', on=['player_id'])
complete_data_2.drop(columns = 'full_name_x',axis =1)
complete_data_2 = complete_data_2.rename({'full_name_y':'full_name'},axis=1)
complete_data_2 = complete_data_2[['last_name',
         'first_name',
         'full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]


In [29]:
stats_cols = ['full_name',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']

complete_data_2 = complete_data_2.drop_duplicates(subset=stats_cols, keep='first')
complete_data_2

last_name       first_name        full_name  player_id  year  \
0          Colon          Bartolo    Bartolo Colon   112526.0  2017   
1         Beltre           Adrian    Adrian Beltre   134181.0  2017   
2        Beltran           Carlos   Carlos Beltran   136860.0  2017   
3          Werth           Jayson     Jayson Werth   150029.0  2017   
4         Arroyo          Bronson   Bronson Arroyo   276520.0  2017   
...          ...              ...              ...        ...   ...   
9777    Gonzalez             Alex    Alex Gonzalez   136460.0  2010   
9778     Edmonds              Jim      Jim Edmonds   113744.0  2010   
9779      Castro             Juan      Juan Castro   112128.0  2010   
10203  Rodriguez  Henry Alejandro  Henry Rodriguez   514719.0  2012   
10204  Rodriguez  Henry Alejandro  Henry Rodriguez   514719.0  2013   

       player_age  b_ab  b_total_pa  b_total_hits  b_single  ...  b_triple  \
0              44    19          20             0         0  ...         0   
1              38   340         389           106        66  ...         1   
2              40   467         509           108        65  ...         0   
3              38   252         289            57        36  ...         1   
4              40    26          29             4         4  ...         0   
...           ...   ...         ...           ...       ...  ...       ...   
9777           33   595         640           149        81  ...         3   
9778           40   246         272            68        34  ...         0   
9779           38   129         140            25        20  ...         0   
10203          22    14          16             3         2  ...         0   
10204          23     9          10             1         1  ...         0   

       b_home_run  b_strikeout  b_walk  b_k_percent  b_bb_percent  \
0               0           10       0         50.0           0.0   
1              17           52      39         13.4          10.0   
2              14          102      33         20.0           6.5   
3              10           69      35         23.9          12.1   
4               0           10       0         34.5           0.0   
...           ...          ...     ...          ...           ...   
9777           23          118      31         18.4           4.8   
9778           11           60      24         22.1           8.8   
9779            0           25       8         17.9           5.7   
10203           0            2       2         12.5          12.5   
10204           0            4       1         40.0          10.0   

       batting_avg  slg_percent  on_base_percent  on_base_plus_slg  
0            0.000        0.000            0.000             0.000  
1            0.312        0.532            0.383             0.915  
2            0.231        0.383            0.283             0.666  
3            0.226        0.393            0.322             0.715  
4            0.154        0.154            0.138             0.292  
...            ...          ...              ...               ...  
9777         0.250        0.447            0.292             0.739  
9778         0.276        0.504            0.342             0.846  
9779         0.194        0.233            0.236             0.468  
10203        0.214        0.286            0.313             0.598  
10204        0.111        0.111            0.200             0.311  

[9781 rows x 21 columns]

In [30]:
complete_data_2 = complete_data_2[['full_name',
         'player_id',
         'year',
         'player_age',
         'b_ab',
         'b_total_pa',
         'b_total_hits',
         'b_single',
         'b_double',
         'b_triple',
         'b_home_run',
         'b_strikeout',
         'b_walk',
         'b_k_percent',
         'b_bb_percent',
         'batting_avg',
         'slg_percent',
         'on_base_percent',
         'on_base_plus_slg']]

In [31]:
print(frame.shape)
print(complete_data_2.shape)

(9811, 21)
(9781, 19)


Great! Now we have the same amount of rows as our initial dataframe!

#### Now let's check for if we have different players with the same name (by same name we mean the same name in our dataframe)

In [32]:
fr_gb = frame.groupby('full_name')['player_id'].nunique().reset_index()
fr_gb = fr_gb.sort_values(by = 'player_id', ascending = False)
fr_gb

full_name  player_id
599    Daniel Robertson          2
0          A.J. Burnett          1
1107    Jeff Samardzija          1
1121  Jeremy Hellickson          1
1994         Ryan Court          1
...                 ...        ...
1165         Joe Mather          0
1164        Joe Mahoney          0
1163      Joe Koshansky          0
1162          Joe Kelly          0
2412      Zoilo Almonte          0

[2413 rows x 2 columns]

We must check what is happening to Daniel Robertson. But first, let's do the same check on the newer database:

In [33]:
cd2_gb = complete_data_2.groupby('full_name')['player_id'].nunique().reset_index()
cd2_gb = cd2_gb.sort_values(by = 'player_id', ascending = False)
cd2_gb

full_name  player_id
1494        Luis Jimenez          2
599     Daniel Robertson          2
29            Adam Eaton          2
1486       Luis Castillo          2
730       Edgar Gonzalez          2
...                  ...        ...
809      Francisco Arcia          1
810   Francisco Cervelli          1
811     Francisco Lindor          1
812    Francisco Liriano          1
2412       Zoilo Almonte          1

[2413 rows x 2 columns]

Here we must check for the players names that have 2 ids.

In [34]:
names = cd2_gb.loc[cd2_gb['player_id']!=1]
names.sort_values(by = 'full_name')

full_name  player_id
29          Adam Eaton          2
310     Brian Anderson          2
449       Chris Carter          2
489        Chris Young          2
599   Daniel Robertson          2
730     Edgar Gonzalez          2
1296         Josh Bell          2
1486     Luis Castillo          2
1491     Luis Gonzalez          2
1494      Luis Jimenez          2

In [35]:
names_list = names['full_name'].to_list()
names_list

['Luis Jimenez',
 'Daniel Robertson',
 'Adam Eaton',
 'Luis Castillo',
 'Edgar Gonzalez',
 'Josh Bell',
 'Luis Gonzalez',
 'Chris Young',
 'Chris Carter',
 'Brian Anderson']

In [36]:
print(fr_gb.shape)
print(cd2_gb.shape)

(2413, 2)
(2413, 2)


We can observe that our datasets have the same amount of unique names.

We can check in the total batters data if these names have indeed 2 ids each (meaning different players with the same name):

In [37]:
bat_data_war['full_name'] = bat_data_war['full_name'].apply(unidecode)
bat_data_war_gb = bat_data_war.groupby('full_name')['player_id'].nunique().reset_index()
bat_data_war_gb.loc[bat_data_war_gb['full_name'].isin(names_list)]

full_name  player_id
57          Adam Eaton          2
556     Brian Anderson          2
822       Chris Carter          2
889        Chris Young          2
1089  Daniel Robertson          2
1347    Edgar Gonzalez          2
2385         Josh Bell          2
2755     Luis Castillo          2
2763     Luis Gonzalez          2
2766      Luis Jimenez          2

As we can see, all the names on our list have 2 ids, and therefore correspond to two players.

## Adding WAR and League Data

As there are players that are batters and pitchers, we need to get both WAR scores. We'll first start with batters WAR.

In [38]:
bat_data_war.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20223 entries, 3 to 118789
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   full_name           20223 non-null  object 
 1   player_age          20223 non-null  float64
 2   player_id           20223 non-null  float64
 3   player_ID           20223 non-null  object 
 4   year                20223 non-null  int64  
 5   team_ID             20223 non-null  object 
 6   stint_ID            20223 non-null  int64  
 7   league              20223 non-null  object 
 8   b_total_pa          19521 non-null  float64
 9   G                   20223 non-null  int64  
 10  Inn                 20223 non-null  float64
 11  runs_bat            20223 non-null  float64
 12  runs_br             20223 non-null  float64
 13  runs_dp             20223 non-null  float64
 14  runs_field          20223 non-null  float64
 15  runs_infield        20223 non-null  float64
 16  run

We are interested in the 'WAR' and 'league' columns.

In [39]:
war_values_bat = bat_data_war[['full_name','player_id','year','league','WAR']]
war_values_bat

full_name  player_id  year league   WAR
3       David Aardsma   430911.0  2008     AL -0.02
4       David Aardsma   430911.0  2009     AL  0.00
5       David Aardsma   430911.0  2010     AL  0.00
6       David Aardsma   430911.0  2012     AL   NaN
7       David Aardsma   430911.0  2013     NL  0.00
...               ...        ...   ...    ...   ...
118754    Mike Zunino   572287.0  2020     AL -0.05
118755    Mike Zunino   572287.0  2021     AL  3.75
118787      Tony Zych   543964.0  2015     AL   NaN
118788      Tony Zych   543964.0  2016     AL   NaN
118789      Tony Zych   543964.0  2017     AL  0.00

[20223 rows x 5 columns]

Let's rename the WAR column to avoid future conflict.

In [40]:
war_values_bat = war_values_bat.rename({'WAR':'WAR_bat'},axis = 1)
war_values_bat

full_name  player_id  year league  WAR_bat
3       David Aardsma   430911.0  2008     AL    -0.02
4       David Aardsma   430911.0  2009     AL     0.00
5       David Aardsma   430911.0  2010     AL     0.00
6       David Aardsma   430911.0  2012     AL      NaN
7       David Aardsma   430911.0  2013     NL     0.00
...               ...        ...   ...    ...      ...
118754    Mike Zunino   572287.0  2020     AL    -0.05
118755    Mike Zunino   572287.0  2021     AL     3.75
118787      Tony Zych   543964.0  2015     AL      NaN
118788      Tony Zych   543964.0  2016     AL      NaN
118789      Tony Zych   543964.0  2017     AL     0.00

[20223 rows x 5 columns]

Now, let's get the pitchers WAR

In [41]:
pit_data_war = pd.read_csv('war_daily_pitch.txt', sep=",", encoding = 'utf8')
pit_data_war = pit_data_war[pit_data_war['year_ID']>=2008]

In [42]:
pit_data_war = pit_data_war.rename({'mlb_ID': 'player_id', 'name_common': 'full_name', 'year_ID': 'year', 'age':'player_age','PA':'b_total_pa','lg_ID':'league'}, axis = 1)

In [43]:
war_values_pit = pit_data_war[['full_name','player_id','year','league','WAR']]

In [44]:
war_values_pit = war_values_pit.rename({'WAR':'WAR_pit'},axis = 1)
war_values_pit

full_name  player_id  year league  WAR_pit
41725        David Aardsma   430911.0  2008     AL    -0.31
41726       Jeremy Accardo   435618.0  2008     AL    -0.52
41727       Alfredo Aceves   469686.0  2008     AL     1.34
41728        Nick Adenhart   444440.0  2008     AL    -0.38
41729  Jonathan Albaladejo   452293.0  2008     AL     0.21
...                    ...        ...   ...    ...      ...
52854      Jordan Yamamoto   657141.0  2021     NL    -0.22
52855         Eric Yardley   642233.0  2021     NL    -0.43
52856         Huascar Ynoa   660623.0  2021     NL     1.45
52857           Alex Young   622065.0  2021     NL    -0.86
52858    Jordan Zimmermann   519455.0  2021     NL    -0.08

[11134 rows x 5 columns]

Now, let's merge the two dataframes

In [45]:
war_values_merged = pd.merge(war_values_bat, war_values_pit, how = 'left', on=['full_name','player_id','year','league'])

In [46]:
war_values_merged

full_name  player_id  year league  WAR_bat  WAR_pit
0      David Aardsma   430911.0  2008     AL    -0.02    -0.31
1      David Aardsma   430911.0  2009     AL     0.00     1.69
2      David Aardsma   430911.0  2010     AL     0.00     0.68
3      David Aardsma   430911.0  2012     AL      NaN    -0.02
4      David Aardsma   430911.0  2013     NL     0.00    -0.13
...              ...        ...   ...    ...      ...      ...
21183    Mike Zunino   572287.0  2020     AL    -0.05      NaN
21184    Mike Zunino   572287.0  2021     AL     3.75      NaN
21185      Tony Zych   543964.0  2015     AL      NaN     0.42
21186      Tony Zych   543964.0  2016     AL      NaN     0.12
21187      Tony Zych   543964.0  2017     AL     0.00     1.07

[21188 rows x 6 columns]

Some players play exclusively in one position, so they will not have WAR results for the position they don't play.

In [47]:
war_values_merged.isnull().sum()

full_name       0
player_id       0
year            0
league          0
WAR_bat      2655
WAR_pit      9360
dtype: int64

In [48]:
war_values_merged = war_values_merged.fillna(0)

Now we can calculate a Total WAR value by adding the two WAR values

In [49]:
war_values_merged['Total_WAR'] = war_values_merged['WAR_bat'] + war_values_merged['WAR_pit']
war_values_merged

full_name  player_id  year league  WAR_bat  WAR_pit  Total_WAR
0      David Aardsma   430911.0  2008     AL    -0.02    -0.31      -0.33
1      David Aardsma   430911.0  2009     AL     0.00     1.69       1.69
2      David Aardsma   430911.0  2010     AL     0.00     0.68       0.68
3      David Aardsma   430911.0  2012     AL     0.00    -0.02      -0.02
4      David Aardsma   430911.0  2013     NL     0.00    -0.13      -0.13
...              ...        ...   ...    ...      ...      ...        ...
21183    Mike Zunino   572287.0  2020     AL    -0.05     0.00      -0.05
21184    Mike Zunino   572287.0  2021     AL     3.75     0.00       3.75
21185      Tony Zych   543964.0  2015     AL     0.00     0.42       0.42
21186      Tony Zych   543964.0  2016     AL     0.00     0.12       0.12
21187      Tony Zych   543964.0  2017     AL     0.00     1.07       1.07

[21188 rows x 7 columns]

In [50]:
war_values_merged.isnull().sum()

full_name    0
player_id    0
year         0
league       0
WAR_bat      0
WAR_pit      0
Total_WAR    0
dtype: int64

### Add the WAR and league results to our dataset

In [51]:
complete_data_and_war = pd.merge(complete_data_2, war_values_merged, how = 'left', on=['full_name','player_id','year'])
complete_data_and_war.columns

complete_data_and_war = complete_data_and_war[['full_name', 'player_id', 'year', 'player_age','league',
       'Total_WAR','WAR_bat', 'WAR_pit', 'b_ab', 'b_total_pa',
       'b_total_hits', 'b_single', 'b_double', 'b_triple', 'b_home_run',
       'b_strikeout', 'b_walk', 'b_k_percent', 'b_bb_percent', 'batting_avg',
       'slg_percent', 'on_base_percent', 'on_base_plus_slg']]
complete_data_and_war

full_name  player_id  year  player_age league  Total_WAR  \
0        Bartolo Colon   112526.0  2017          44     NL      -2.35   
1        Bartolo Colon   112526.0  2017          44     AL       0.21   
2        Adrian Beltre   134181.0  2017          38     AL       3.78   
3       Carlos Beltran   136860.0  2017          40     AL      -0.79   
4         Jayson Werth   150029.0  2017          38     NL      -0.70   
...                ...        ...   ...         ...    ...        ...   
10894      Jim Edmonds   113744.0  2010          40     NL       1.98   
10895      Juan Castro   112128.0  2010          38     NL      -0.13   
10896      Juan Castro   112128.0  2010          38     NL      -1.75   
10897  Henry Rodriguez   514719.0  2012          22     NL      -0.04   
10898  Henry Rodriguez   514719.0  2013          23     NL      -0.11   

       WAR_bat  WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  \
0        -0.22    -2.13    19          20  ...         0           0   
1        -0.07     0.28    19          20  ...         0           0   
2         3.78     0.00   340         389  ...         1          17   
3        -0.79     0.00   467         509  ...         0          14   
4        -0.70     0.00   252         289  ...         1          10   
...        ...      ...   ...         ...  ...       ...         ...   
10894     1.98     0.00   246         272  ...         0          11   
10895    -0.13     0.00   129         140  ...         0           0   
10896    -1.75     0.00   129         140  ...         0           0   
10897    -0.04     0.00    14          16  ...         0           0   
10898    -0.11     0.00     9          10  ...         0           0   

       b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0               10       0         50.0           0.0        0.000   
1               10       0         50.0           0.0        0.000   
2               52      39         13.4          10.0        0.312   
3              102      33         20.0           6.5        0.231   
4               69      35         23.9          12.1        0.226   
...            ...     ...          ...           ...          ...   
10894           60      24         22.1           8.8        0.276   
10895           25       8         17.9           5.7        0.194   
10896           25       8         17.9           5.7        0.194   
10897            2       2         12.5          12.5        0.214   
10898            4       1         40.0          10.0        0.111   

       slg_percent  on_base_percent  on_base_plus_slg  
0            0.000            0.000             0.000  
1            0.000            0.000             0.000  
2            0.532            0.383             0.915  
3            0.383            0.283             0.666  
4            0.393            0.322             0.715  
...            ...              ...               ...  
10894        0.504            0.342             0.846  
10895        0.233            0.236             0.468  
10896        0.233            0.236             0.468  
10897        0.286            0.313             0.598  
10898        0.111            0.200             0.311  

[10899 rows x 23 columns]

In [52]:
complete_data_and_war.drop_duplicates()

full_name  player_id  year  player_age league  Total_WAR  \
0        Bartolo Colon   112526.0  2017          44     NL      -2.35   
1        Bartolo Colon   112526.0  2017          44     AL       0.21   
2        Adrian Beltre   134181.0  2017          38     AL       3.78   
3       Carlos Beltran   136860.0  2017          40     AL      -0.79   
4         Jayson Werth   150029.0  2017          38     NL      -0.70   
...                ...        ...   ...         ...    ...        ...   
10894      Jim Edmonds   113744.0  2010          40     NL       1.98   
10895      Juan Castro   112128.0  2010          38     NL      -0.13   
10896      Juan Castro   112128.0  2010          38     NL      -1.75   
10897  Henry Rodriguez   514719.0  2012          22     NL      -0.04   
10898  Henry Rodriguez   514719.0  2013          23     NL      -0.11   

       WAR_bat  WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  \
0        -0.22    -2.13    19          20  ...         0           0   
1        -0.07     0.28    19          20  ...         0           0   
2         3.78     0.00   340         389  ...         1          17   
3        -0.79     0.00   467         509  ...         0          14   
4        -0.70     0.00   252         289  ...         1          10   
...        ...      ...   ...         ...  ...       ...         ...   
10894     1.98     0.00   246         272  ...         0          11   
10895    -0.13     0.00   129         140  ...         0           0   
10896    -1.75     0.00   129         140  ...         0           0   
10897    -0.04     0.00    14          16  ...         0           0   
10898    -0.11     0.00     9          10  ...         0           0   

       b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
0               10       0         50.0           0.0        0.000   
1               10       0         50.0           0.0        0.000   
2               52      39         13.4          10.0        0.312   
3              102      33         20.0           6.5        0.231   
4               69      35         23.9          12.1        0.226   
...            ...     ...          ...           ...          ...   
10894           60      24         22.1           8.8        0.276   
10895           25       8         17.9           5.7        0.194   
10896           25       8         17.9           5.7        0.194   
10897            2       2         12.5          12.5        0.214   
10898            4       1         40.0          10.0        0.111   

       slg_percent  on_base_percent  on_base_plus_slg  
0            0.000            0.000             0.000  
1            0.000            0.000             0.000  
2            0.532            0.383             0.915  
3            0.383            0.283             0.666  
4            0.393            0.322             0.715  
...            ...              ...               ...  
10894        0.504            0.342             0.846  
10895        0.233            0.236             0.468  
10896        0.233            0.236             0.468  
10897        0.286            0.313             0.598  
10898        0.111            0.200             0.311  

[10881 rows x 23 columns]

In [53]:
complete_data_and_war.isnull().sum()

full_name            0
player_id            0
year                 0
player_age           0
league              89
Total_WAR           89
WAR_bat             89
WAR_pit             89
b_ab                 0
b_total_pa           0
b_total_hits         0
b_single             0
b_double             0
b_triple             0
b_home_run           0
b_strikeout          0
b_walk               0
b_k_percent          0
b_bb_percent         0
batting_avg          0
slg_percent          0
on_base_percent      0
on_base_plus_slg     0
dtype: int64

There are some players that played for different team during a same season. To our model, we will pick the highest total war to chose between the data available.

In [54]:
complete_data_and_war = complete_data_and_war.sort_values(by = 'Total_WAR', ascending = False)
complete_data_and_war

full_name  player_id  year  player_age league  Total_WAR  \
1860      Mookie Betts   605141.0  2018          26     AL      10.68   
5394        Mike Trout   545361.0  2012          21     AL      10.54   
3438        Mike Trout   545361.0  2016          25     AL      10.48   
1802      Jacob deGrom   594798.0  2018          30     NL      10.36   
1654        Mike Trout   545361.0  2018          27     AL       9.92   
...                ...        ...   ...         ...    ...        ...   
10319  Liván Hernández   115817.0  2008          33    NaN        NaN   
10366  Yoenis Céspedes   493316.0  2014          29    NaN        NaN   
10470    José Iglesias   578428.0  2013          23    NaN        NaN   
10650   José Contreras   425747.0  2009          38    NaN        NaN   
10742  Liván Hernández   115817.0  2009          34    NaN        NaN   

       WAR_bat  WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  \
1860     10.68      0.0   520         614  ...         5          32   
5394     10.54      0.0   559         639  ...         8          30   
3438     10.48      0.0   549         681  ...         5          29   
1802      0.46      9.9    67          74  ...         0           0   
1654      9.92      0.0   471         608  ...         4          39   
...        ...      ...   ...         ...  ...       ...         ...   
10319      NaN      NaN    19          22  ...         0           0   
10366      NaN      NaN   600         645  ...         6          22   
10470      NaN      NaN   350         382  ...         2           3   
10650      NaN      NaN     9          10  ...         0           0   
10742      NaN      NaN    54          67  ...         0           0   

       b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
1860            91      81         14.8          13.2        0.346   
5394           139      67         21.8          10.5        0.326   
3438           137     116         20.1          17.0        0.315   
1802            25       4         33.8           5.4        0.164   
1654           124     122         20.4          20.1        0.312   
...            ...     ...          ...           ...          ...   
10319            1       0          4.5           0.0        0.263   
10366          128      35         19.8           5.4        0.260   
10470           60      15         15.7           3.9        0.303   
10650            5       0         50.0           0.0        0.000   
10742            6       2          9.0           3.0        0.148   

       slg_percent  on_base_percent  on_base_plus_slg  
1860         0.640            0.438             1.078  
5394         0.564            0.399             0.963  
3438         0.550            0.441             0.991  
1802         0.179            0.203             0.382  
1654         0.628            0.459             1.087  
...            ...              ...               ...  
10319        0.263            0.227             0.490  
10366        0.450            0.301             0.751  
10470        0.386            0.346             0.731  
10650        0.000            0.000             0.000  
10742        0.185            0.149             0.334  

[10899 rows x 23 columns]

In [55]:
complete_data_and_war = complete_data_and_war.drop_duplicates(subset=['full_name','player_id','year'], keep='first')
complete_data_and_war

full_name  player_id  year  player_age league  Total_WAR  \
1860      Mookie Betts   605141.0  2018          26     AL      10.68   
5394        Mike Trout   545361.0  2012          21     AL      10.54   
3438        Mike Trout   545361.0  2016          25     AL      10.48   
1802      Jacob deGrom   594798.0  2018          30     NL      10.36   
1654        Mike Trout   545361.0  2018          27     AL       9.92   
...                ...        ...   ...         ...    ...        ...   
10319  Liván Hernández   115817.0  2008          33    NaN        NaN   
10366  Yoenis Céspedes   493316.0  2014          29    NaN        NaN   
10470    José Iglesias   578428.0  2013          23    NaN        NaN   
10650   José Contreras   425747.0  2009          38    NaN        NaN   
10742  Liván Hernández   115817.0  2009          34    NaN        NaN   

       WAR_bat  WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  \
1860     10.68      0.0   520         614  ...         5          32   
5394     10.54      0.0   559         639  ...         8          30   
3438     10.48      0.0   549         681  ...         5          29   
1802      0.46      9.9    67          74  ...         0           0   
1654      9.92      0.0   471         608  ...         4          39   
...        ...      ...   ...         ...  ...       ...         ...   
10319      NaN      NaN    19          22  ...         0           0   
10366      NaN      NaN   600         645  ...         6          22   
10470      NaN      NaN   350         382  ...         2           3   
10650      NaN      NaN     9          10  ...         0           0   
10742      NaN      NaN    54          67  ...         0           0   

       b_strikeout  b_walk  b_k_percent  b_bb_percent  batting_avg  \
1860            91      81         14.8          13.2        0.346   
5394           139      67         21.8          10.5        0.326   
3438           137     116         20.1          17.0        0.315   
1802            25       4         33.8           5.4        0.164   
1654           124     122         20.4          20.1        0.312   
...            ...     ...          ...           ...          ...   
10319            1       0          4.5           0.0        0.263   
10366          128      35         19.8           5.4        0.260   
10470           60      15         15.7           3.9        0.303   
10650            5       0         50.0           0.0        0.000   
10742            6       2          9.0           3.0        0.148   

       slg_percent  on_base_percent  on_base_plus_slg  
1860         0.640            0.438             1.078  
5394         0.564            0.399             0.963  
3438         0.550            0.441             0.991  
1802         0.179            0.203             0.382  
1654         0.628            0.459             1.087  
...            ...              ...               ...  
10319        0.263            0.227             0.490  
10366        0.450            0.301             0.751  
10470        0.386            0.346             0.731  
10650        0.000            0.000             0.000  
10742        0.185            0.149             0.334  

[9781 rows x 23 columns]

In [56]:
complete_data_and_war.isnull().sum()

full_name            0
player_id            0
year                 0
player_age           0
league              89
Total_WAR           89
WAR_bat             89
WAR_pit             89
b_ab                 0
b_total_pa           0
b_total_hits         0
b_single             0
b_double             0
b_triple             0
b_home_run           0
b_strikeout          0
b_walk               0
b_k_percent          0
b_bb_percent         0
batting_avg          0
slg_percent          0
on_base_percent      0
on_base_plus_slg     0
dtype: int64

### Adding the MVP Votes and Ranking

In [57]:
for year in range(2008,2022):

    NL_df = pd.read_csv(f'MVP Votes Raw Data/NL_{year}.csv')
    AL_df = pd.read_csv(f'MVP Votes Raw Data/AL_{year}.csv')

    ref = [NL_df, AL_df]
    df_comb = pd.concat(ref)
    cols = ['first_name', 'last_name']
    df_comb['full_name'] = df_comb[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    df_comb['year'] = year
    df_comb.sort_values(by = 'rank')
    
    df_comb.to_csv(f'MVP Cleaned Data/{year}_mvp.csv')


In [58]:
path = 'MVP Cleaned Data'
all_files = glob.glob(path + "/*_mvp.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

voted_for_mvps = pd.concat(li, axis=0, ignore_index=True)

In [59]:
voted_for_mvps = voted_for_mvps[['full_name','year','rank','vote_points','1st_place','mvp_share']]

We are considering a timeframe of 14 years. Therefore, we should expect 28 players ranked as 1st. 

In [60]:
voted_for_mvps.groupby('rank')['rank'].count()

rank
1     28
2     28
3     28
4     28
5     28
6     29
7     28
8     27
9     29
10    28
11    27
12    30
13    29
14    30
15    33
16    28
17    33
18    27
19    30
20    35
21    23
22    19
23    14
24    12
25     9
26     7
27     1
28    11
30     2
32     4
Name: rank, dtype: int64

In [61]:
stats_war_mvp_data = pd.merge(complete_data_and_war, voted_for_mvps, how = 'left', on=['full_name','year'])
stats_war_mvp_data = stats_war_mvp_data[['full_name', 'player_id', 'year', 'player_age', 'league', 'rank', 'vote_points',
       '1st_place', 'mvp_share','Total_WAR','WAR_bat', 'WAR_pit', 'b_ab', 'b_total_pa', 'b_total_hits', 'b_single',
       'b_double', 'b_triple', 'b_home_run', 'b_strikeout', 'b_walk',
       'b_k_percent', 'b_bb_percent', 'batting_avg', 'slg_percent',
       'on_base_percent', 'on_base_plus_slg']]
stats_war_mvp_data

full_name  player_id  year  player_age league  rank  vote_points  \
0        Mookie Betts   605141.0  2018          26     AL   1.0        410.0   
1          Mike Trout   545361.0  2012          21     AL   2.0        281.0   
2          Mike Trout   545361.0  2016          25     AL   1.0        356.0   
3        Jacob deGrom   594798.0  2018          30     NL   5.0        141.0   
4          Mike Trout   545361.0  2018          27     AL   2.0        265.0   
...               ...        ...   ...         ...    ...   ...          ...   
9776  Liván Hernández   115817.0  2008          33    NaN   NaN          NaN   
9777  Yoenis Céspedes   493316.0  2014          29    NaN   NaN          NaN   
9778    José Iglesias   578428.0  2013          23    NaN   NaN          NaN   
9779   José Contreras   425747.0  2009          38    NaN   NaN          NaN   
9780  Liván Hernández   115817.0  2009          34    NaN   NaN          NaN   

      1st_place  mvp_share  Total_WAR  ...  b_triple  b_home_run  b_strikeout  \
0          28.0       0.98      10.68  ...         5          32           91   
1           6.0       0.72      10.54  ...         8          30          139   
2          19.0       0.85      10.48  ...         5          29          137   
3           1.0       0.34      10.36  ...         0           0           25   
4           1.0       0.63       9.92  ...         4          39          124   
...         ...        ...        ...  ...       ...         ...          ...   
9776        NaN        NaN        NaN  ...         0           0            1   
9777        NaN        NaN        NaN  ...         6          22          128   
9778        NaN        NaN        NaN  ...         2           3           60   
9779        NaN        NaN        NaN  ...         0           0            5   
9780        NaN        NaN        NaN  ...         0           0            6   

      b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0         81         14.8          13.2        0.346        0.640   
1         67         21.8          10.5        0.326        0.564   
2        116         20.1          17.0        0.315        0.550   
3          4         33.8           5.4        0.164        0.179   
4        122         20.4          20.1        0.312        0.628   
...      ...          ...           ...          ...          ...   
9776       0          4.5           0.0        0.263        0.263   
9777      35         19.8           5.4        0.260        0.450   
9778      15         15.7           3.9        0.303        0.386   
9779       0         50.0           0.0        0.000        0.000   
9780       2          9.0           3.0        0.148        0.185   

      on_base_percent  on_base_plus_slg  
0               0.438             1.078  
1               0.399             0.963  
2               0.441             0.991  
3               0.203             0.382  
4               0.459             1.087  
...               ...               ...  
9776            0.227             0.490  
9777            0.301             0.751  
9778            0.346             0.731  
9779            0.000             0.000  
9780            0.149             0.334  

[9781 rows x 27 columns]

However, our stats dataset only accounts for batters. During this timeframe, only one pitcher has been awarded as MVP, in 2011. Therefore, we should find 27 #1 in our set. Let's check:

In [62]:
stats_war_mvp_data.groupby('rank')['rank'].count()

rank
1.0     27
2.0     28
3.0     26
4.0     27
5.0     27
6.0     27
7.0     27
8.0     24
9.0     26
10.0    23
11.0    23
12.0    25
13.0    24
14.0    24
15.0    27
16.0    24
17.0    27
18.0    21
19.0    22
20.0    32
21.0    19
22.0    12
23.0    13
24.0    11
25.0     7
26.0     6
27.0     1
28.0     9
30.0     2
32.0     4
Name: rank, dtype: int64

Now, let's assign 0 for all players that haven't been voted.

In [63]:
stats_war_mvp_data['rank'] = stats_war_mvp_data['rank'].fillna(0)
stats_war_mvp_data['vote_points'] = stats_war_mvp_data['vote_points'].fillna(0)
stats_war_mvp_data['1st_place'] = stats_war_mvp_data['1st_place'].fillna(0)
stats_war_mvp_data['mvp_share'] = stats_war_mvp_data['mvp_share'].fillna(0)

In [64]:
stats_war_mvp_data

full_name  player_id  year  player_age league  rank  vote_points  \
0        Mookie Betts   605141.0  2018          26     AL   1.0        410.0   
1          Mike Trout   545361.0  2012          21     AL   2.0        281.0   
2          Mike Trout   545361.0  2016          25     AL   1.0        356.0   
3        Jacob deGrom   594798.0  2018          30     NL   5.0        141.0   
4          Mike Trout   545361.0  2018          27     AL   2.0        265.0   
...               ...        ...   ...         ...    ...   ...          ...   
9776  Liván Hernández   115817.0  2008          33    NaN   0.0          0.0   
9777  Yoenis Céspedes   493316.0  2014          29    NaN   0.0          0.0   
9778    José Iglesias   578428.0  2013          23    NaN   0.0          0.0   
9779   José Contreras   425747.0  2009          38    NaN   0.0          0.0   
9780  Liván Hernández   115817.0  2009          34    NaN   0.0          0.0   

      1st_place  mvp_share  Total_WAR  ...  b_triple  b_home_run  b_strikeout  \
0          28.0       0.98      10.68  ...         5          32           91   
1           6.0       0.72      10.54  ...         8          30          139   
2          19.0       0.85      10.48  ...         5          29          137   
3           1.0       0.34      10.36  ...         0           0           25   
4           1.0       0.63       9.92  ...         4          39          124   
...         ...        ...        ...  ...       ...         ...          ...   
9776        0.0       0.00        NaN  ...         0           0            1   
9777        0.0       0.00        NaN  ...         6          22          128   
9778        0.0       0.00        NaN  ...         2           3           60   
9779        0.0       0.00        NaN  ...         0           0            5   
9780        0.0       0.00        NaN  ...         0           0            6   

      b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0         81         14.8          13.2        0.346        0.640   
1         67         21.8          10.5        0.326        0.564   
2        116         20.1          17.0        0.315        0.550   
3          4         33.8           5.4        0.164        0.179   
4        122         20.4          20.1        0.312        0.628   
...      ...          ...           ...          ...          ...   
9776       0          4.5           0.0        0.263        0.263   
9777      35         19.8           5.4        0.260        0.450   
9778      15         15.7           3.9        0.303        0.386   
9779       0         50.0           0.0        0.000        0.000   
9780       2          9.0           3.0        0.148        0.185   

      on_base_percent  on_base_plus_slg  
0               0.438             1.078  
1               0.399             0.963  
2               0.441             0.991  
3               0.203             0.382  
4               0.459             1.087  
...               ...               ...  
9776            0.227             0.490  
9777            0.301             0.751  
9778            0.346             0.731  
9779            0.000             0.000  
9780            0.149             0.334  

[9781 rows x 27 columns]

## Final data analysis

Now, before closing the data cleaning, we need to remove some useless data. We can observe that some rows don't have WAR information, which is a crucial piece of data for our analysis.
Therefore, we shall drop the rows without this information, as we already added our WAR values.

In [65]:
stats_war_mvp_data = stats_war_mvp_data[stats_war_mvp_data['Total_WAR'].notna()]
stats_war_mvp_data.isnull().sum()

full_name           0
player_id           0
year                0
player_age          0
league              0
rank                0
vote_points         0
1st_place           0
mvp_share           0
Total_WAR           0
WAR_bat             0
WAR_pit             0
b_ab                0
b_total_pa          0
b_total_hits        0
b_single            0
b_double            0
b_triple            0
b_home_run          0
b_strikeout         0
b_walk              0
b_k_percent         0
b_bb_percent        0
batting_avg         0
slg_percent         0
on_base_percent     0
on_base_plus_slg    0
dtype: int64

Great! Nothing missing!

### Spliting the final dataset into separated datasets

In [66]:
for year in range(2008,2022):
    
    export_data = stats_war_mvp_data.loc[stats_war_mvp_data['year'] == year]
    #export_data.to_csv(f'./Complete/{year}_complete.csv')

## Adding the 2022 players available information

In [67]:
new_year = 2022

df_AL = pd.read_csv(f'Statcast Raw Data/{new_year}_AL.csv')
df_NL = pd.read_csv(f'Statcast Raw Data/{new_year}_NL.csv')

From these Datasets, we don't have much relevant data. Let's filter our data:

In [68]:
df_AL = df_AL[['player_name','player_id','abs','hits']]
df_NL = df_NL[['player_name','player_id','abs','hits']]

### 2022 AL League players age finder

In [69]:
df_AL

player_name  player_id  abs  hits
0         Murphy, Tom     608596    3     2
1        Kwan, Steven     680757   15    10
2        Choi, Ji-Man     596847   13     8
3         Pollock, AJ     572041    7     4
4       Jansen, Danny     643376    7     4
..                ...        ...  ...   ...
192     Sheets, Gavin     657757    6     0
193  Gonzalez, Marwin     503556    1     0
194      Smith, Kevin     675656   12     0
195         Chang, Yu     644374    7     0
196      Moore, Dylan     664238    0     0

[197 rows x 4 columns]

#### Using the bat war dataset as out players index

In [70]:
player_id_name_age = bat_data_war[['full_name','player_id','player_age','year']]
player_id_name_age = player_id_name_age.rename({'player_age':'player_age_war_db','year':'year_war_db'}, axis=1)
player_id_name_age = player_id_name_age.sort_values(by = 'player_age_war_db', ascending = False)
player_id_name_age2 = player_id_name_age.drop_duplicates(subset = 'player_id', keep = 'first')
player_id_name_age2

full_name  player_id  player_age_war_db  year_war_db
75367       Jamie Moyer   119469.0               49.0         2012
53818     Randy Johnson   116615.0               45.0         2009
104201    Ichiro Suzuki   400085.0               45.0         2019
110393     Omar Vizquel   123744.0               45.0         2012
20868     Bartolo Colon   112526.0               45.0         2018
...                 ...        ...                ...          ...
66211   Brailyn Marquez   666622.0               21.0         2020
81896       Luis Patino   672715.0               20.0         2020
75729      Andres Munoz   662253.0               20.0         2019
35162     Wander Franco   677551.0               20.0         2021
63971     Elvis Luciano   672773.0               19.0         2019

[4426 rows x 4 columns]

In [71]:
player_age_lookup_merge_AL = pd.merge(df_AL, player_id_name_age2, how = 'left', on=['player_id'])
player_age_lookup_merge_AL

player_name  player_id  abs  hits        full_name  \
0         Murphy, Tom     608596    3     2       Tom Murphy   
1        Kwan, Steven     680757   15    10              NaN   
2        Choi, Ji-Man     596847   13     8      Ji-Man Choi   
3         Pollock, AJ     572041    7     4       AJ Pollock   
4       Jansen, Danny     643376    7     4     Danny Jansen   
..                ...        ...  ...   ...              ...   
192     Sheets, Gavin     657757    6     0     Gavin Sheets   
193  Gonzalez, Marwin     503556    1     0  Marwin Gonzalez   
194      Smith, Kevin     675656   12     0      Kevin Smith   
195         Chang, Yu     644374    7     0         Yu Chang   
196      Moore, Dylan     664238    0     0      Dylan Moore   

     player_age_war_db  year_war_db  
0                 30.0       2021.0  
1                  NaN          NaN  
2                 30.0       2021.0  
3                 33.0       2021.0  
4                 26.0       2021.0  
..                 ...          ...  
192               25.0       2021.0  
193               32.0       2021.0  
194               24.0       2021.0  
195               25.0       2021.0  
196               28.0       2021.0  

[197 rows x 7 columns]

In [72]:
player_age_lookup_merge_AL['year'] = new_year
player_age_lookup_merge_AL['years_diff'] = player_age_lookup_merge_AL['year']-player_age_lookup_merge_AL['year_war_db']
player_age_lookup_merge_AL['player_age'] = player_age_lookup_merge_AL['player_age_war_db'] + player_age_lookup_merge_AL['years_diff']
player_age_lookup_merge_AL

player_name  player_id  abs  hits        full_name  \
0         Murphy, Tom     608596    3     2       Tom Murphy   
1        Kwan, Steven     680757   15    10              NaN   
2        Choi, Ji-Man     596847   13     8      Ji-Man Choi   
3         Pollock, AJ     572041    7     4       AJ Pollock   
4       Jansen, Danny     643376    7     4     Danny Jansen   
..                ...        ...  ...   ...              ...   
192     Sheets, Gavin     657757    6     0     Gavin Sheets   
193  Gonzalez, Marwin     503556    1     0  Marwin Gonzalez   
194      Smith, Kevin     675656   12     0      Kevin Smith   
195         Chang, Yu     644374    7     0         Yu Chang   
196      Moore, Dylan     664238    0     0      Dylan Moore   

     player_age_war_db  year_war_db  year  years_diff  player_age  
0                 30.0       2021.0  2022         1.0        31.0  
1                  NaN          NaN  2022         NaN         NaN  
2                 30.0       2021.0  2022         1.0        31.0  
3                 33.0       2021.0  2022         1.0        34.0  
4                 26.0       2021.0  2022         1.0        27.0  
..                 ...          ...   ...         ...         ...  
192               25.0       2021.0  2022         1.0        26.0  
193               32.0       2021.0  2022         1.0        33.0  
194               24.0       2021.0  2022         1.0        25.0  
195               25.0       2021.0  2022         1.0        26.0  
196               28.0       2021.0  2022         1.0        29.0  

[197 rows x 10 columns]

In [73]:
player_age_lookup_merge_AL_nan = player_age_lookup_merge_AL[player_age_lookup_merge_AL['player_age_war_db'].isna()]
player_age_lookup_merge_AL_nan


list_of_pl_AL = player_age_lookup_merge_AL_nan[['player_id','player_name']]
list_of_pl_AL

player_id         player_name
1       680757        Kwan, Steven
56      665161        Peña, Jeremy
143     677951     Witt Jr., Bobby
154     679529  Torkelson, Spencer
157     677594    Rodríguez, Julio
170     669309          Perez, Joe
187     681526    Lavastida, Bryan

In [74]:
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 680757, 'full_name'] = 'Steven Kwan'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 680757, 'player_age'] = 25

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 665161, 'full_name'] = 'Jeremy Pena'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 665161, 'player_age'] = 25

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 677951, 'full_name'] = 'Bobby Witt Jr.'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 677951, 'player_age'] = 22

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 679529, 'full_name'] = 'Spencer Torkelson'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 679529, 'player_age'] = 23

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 677594, 'full_name'] = 'Julio Rodriguez'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 677594, 'player_age'] = 22

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 669309, 'full_name'] = 'Joe Perez'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 669309, 'player_age'] = 23

player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 681526, 'full_name'] = 'Bryan Lavastida'
player_age_lookup_merge_AL.loc[player_age_lookup_merge_AL['player_id'] == 681526, 'player_age'] = 24

player_age_lookup_merge_AL_final = player_age_lookup_merge_AL[['full_name','player_id','year','player_age']]
player_age_lookup_merge_AL_final['league'] = 'AL'
player_age_lookup_merge_AL_final

/var/folders/d8/89h35qn507zfzwn9__35p_bc0000gn/T/ipykernel_68308/3214079982.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_age_lookup_merge_AL_final['league'] = 'AL'


full_name  player_id  year  player_age league
0         Tom Murphy     608596  2022        31.0     AL
1        Steven Kwan     680757  2022        25.0     AL
2        Ji-Man Choi     596847  2022        31.0     AL
3         AJ Pollock     572041  2022        34.0     AL
4       Danny Jansen     643376  2022        27.0     AL
..               ...        ...   ...         ...    ...
192     Gavin Sheets     657757  2022        26.0     AL
193  Marwin Gonzalez     503556  2022        33.0     AL
194      Kevin Smith     675656  2022        25.0     AL
195         Yu Chang     644374  2022        26.0     AL
196      Dylan Moore     664238  2022        29.0     AL

[197 rows x 5 columns]

In [75]:
list_of_cols = ['full_name', 'player_id', 'year', 'player_age','league',
       'Total_WAR','WAR_bat', 'WAR_pit', 'b_ab', 'b_total_pa',
       'b_total_hits', 'b_single', 'b_double', 'b_triple', 'b_home_run',
       'b_strikeout', 'b_walk', 'b_k_percent', 'b_bb_percent', 'batting_avg',
       'slg_percent', 'on_base_percent', 'on_base_plus_slg']

player_age_lookup_merge_AL_final = player_age_lookup_merge_AL_final.reindex(columns = list_of_cols)
player_age_lookup_merge_AL_final

full_name  player_id  year  player_age league  Total_WAR  WAR_bat  \
0         Tom Murphy     608596  2022        31.0     AL        NaN      NaN   
1        Steven Kwan     680757  2022        25.0     AL        NaN      NaN   
2        Ji-Man Choi     596847  2022        31.0     AL        NaN      NaN   
3         AJ Pollock     572041  2022        34.0     AL        NaN      NaN   
4       Danny Jansen     643376  2022        27.0     AL        NaN      NaN   
..               ...        ...   ...         ...    ...        ...      ...   
192     Gavin Sheets     657757  2022        26.0     AL        NaN      NaN   
193  Marwin Gonzalez     503556  2022        33.0     AL        NaN      NaN   
194      Kevin Smith     675656  2022        25.0     AL        NaN      NaN   
195         Yu Chang     644374  2022        26.0     AL        NaN      NaN   
196      Dylan Moore     664238  2022        29.0     AL        NaN      NaN   

     WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  b_strikeout  \
0        NaN   NaN         NaN  ...       NaN         NaN          NaN   
1        NaN   NaN         NaN  ...       NaN         NaN          NaN   
2        NaN   NaN         NaN  ...       NaN         NaN          NaN   
3        NaN   NaN         NaN  ...       NaN         NaN          NaN   
4        NaN   NaN         NaN  ...       NaN         NaN          NaN   
..       ...   ...         ...  ...       ...         ...          ...   
192      NaN   NaN         NaN  ...       NaN         NaN          NaN   
193      NaN   NaN         NaN  ...       NaN         NaN          NaN   
194      NaN   NaN         NaN  ...       NaN         NaN          NaN   
195      NaN   NaN         NaN  ...       NaN         NaN          NaN   
196      NaN   NaN         NaN  ...       NaN         NaN          NaN   

     b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0       NaN          NaN           NaN          NaN          NaN   
1       NaN          NaN           NaN          NaN          NaN   
2       NaN          NaN           NaN          NaN          NaN   
3       NaN          NaN           NaN          NaN          NaN   
4       NaN          NaN           NaN          NaN          NaN   
..      ...          ...           ...          ...          ...   
192     NaN          NaN           NaN          NaN          NaN   
193     NaN          NaN           NaN          NaN          NaN   
194     NaN          NaN           NaN          NaN          NaN   
195     NaN          NaN           NaN          NaN          NaN   
196     NaN          NaN           NaN          NaN          NaN   

     on_base_percent  on_base_plus_slg  
0                NaN               NaN  
1                NaN               NaN  
2                NaN               NaN  
3                NaN               NaN  
4                NaN               NaN  
..               ...               ...  
192              NaN               NaN  
193              NaN               NaN  
194              NaN               NaN  
195              NaN               NaN  
196              NaN               NaN  

[197 rows x 23 columns]

### 2022 NL League players age finder

In [76]:
df_NL

player_name  player_id  abs  hits
0    De La Cruz, Bryan     650559    1     1
1           Bohm, Alec     664761    4     3
2            Happ, Ian     664023   10     7
3       Barnes, Austin     605131    3     2
4       Daza, Yonathan     602074    4     2
..                 ...        ...  ...   ...
191    Guillorme, Luis     641645    8     0
192     Garcia, Aramis     605244    2     0
193      Herrera, Jose     645444    5     0
194      Henry, Payton     668721    3     0
195   Hernandez, Yonny     660634    0     0

[196 rows x 4 columns]

In [77]:
player_age_lookup_merge_NL = pd.merge(df_NL, player_id_name_age2, how = 'left', on=['player_id'])
player_age_lookup_merge_NL

player_name  player_id  abs  hits         full_name  \
0    De La Cruz, Bryan     650559    1     1  Bryan De La Cruz   
1           Bohm, Alec     664761    4     3         Alec Bohm   
2            Happ, Ian     664023   10     7          Ian Happ   
3       Barnes, Austin     605131    3     2     Austin Barnes   
4       Daza, Yonathan     602074    4     2     Yonathan Daza   
..                 ...        ...  ...   ...               ...   
191    Guillorme, Luis     641645    8     0    Luis Guillorme   
192     Garcia, Aramis     605244    2     0     Aramis Garcia   
193      Herrera, Jose     645444    5     0               NaN   
194      Henry, Payton     668721    3     0      Payton Henry   
195   Hernandez, Yonny     660634    0     0   Yonny Hernandez   

     player_age_war_db  year_war_db  
0                 24.0       2021.0  
1                 24.0       2021.0  
2                 26.0       2021.0  
3                 31.0       2021.0  
4                 27.0       2021.0  
..                 ...          ...  
191               26.0       2021.0  
192               28.0       2021.0  
193                NaN          NaN  
194               24.0       2021.0  
195               23.0       2021.0  

[196 rows x 7 columns]

In [78]:
player_age_lookup_merge_NL['year'] = new_year
player_age_lookup_merge_NL['years_diff'] = player_age_lookup_merge_NL['year']-player_age_lookup_merge_NL['year_war_db']
player_age_lookup_merge_NL['player_age'] = player_age_lookup_merge_NL['player_age_war_db'] + player_age_lookup_merge_NL['years_diff']
player_age_lookup_merge_NL

player_name  player_id  abs  hits         full_name  \
0    De La Cruz, Bryan     650559    1     1  Bryan De La Cruz   
1           Bohm, Alec     664761    4     3         Alec Bohm   
2            Happ, Ian     664023   10     7          Ian Happ   
3       Barnes, Austin     605131    3     2     Austin Barnes   
4       Daza, Yonathan     602074    4     2     Yonathan Daza   
..                 ...        ...  ...   ...               ...   
191    Guillorme, Luis     641645    8     0    Luis Guillorme   
192     Garcia, Aramis     605244    2     0     Aramis Garcia   
193      Herrera, Jose     645444    5     0               NaN   
194      Henry, Payton     668721    3     0      Payton Henry   
195   Hernandez, Yonny     660634    0     0   Yonny Hernandez   

     player_age_war_db  year_war_db  year  years_diff  player_age  
0                 24.0       2021.0  2022         1.0        25.0  
1                 24.0       2021.0  2022         1.0        25.0  
2                 26.0       2021.0  2022         1.0        27.0  
3                 31.0       2021.0  2022         1.0        32.0  
4                 27.0       2021.0  2022         1.0        28.0  
..                 ...          ...   ...         ...         ...  
191               26.0       2021.0  2022         1.0        27.0  
192               28.0       2021.0  2022         1.0        29.0  
193                NaN          NaN  2022         NaN         NaN  
194               24.0       2021.0  2022         1.0        25.0  
195               23.0       2021.0  2022         1.0        24.0  

[196 rows x 10 columns]

In [79]:
player_age_lookup_merge_NL_nan = player_age_lookup_merge_NL[player_age_lookup_merge_NL['player_age_war_db'].isna()]
player_age_lookup_merge_NL_nan


list_of_pl_NL = player_age_lookup_merge_NL_nan[['player_id','player_name']]
list_of_pl_NL

player_id       player_name
7       671218     Ramos, Heliot
12      640492      Azocar, José
21      673548     Suzuki, Seiya
24      660636   Castillo, Diego
113     681082     Stott, Bryson
158     669450    Hummel, Cooper
159     682928      Abrams, C.J.
175     665922  Muzziotti, Símon
189     665650       Fox, Lucius
193     645444     Herrera, Jose

In [80]:
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 671218, 'full_name'] = 'Heliot Ramos'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 671218, 'player_age'] = 23

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 640492, 'full_name'] = 'Jose Azocar'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 640492, 'player_age'] = 26

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 673548, 'full_name'] = 'Seiya Suzuki'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 673548, 'player_age'] = 28

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 681082, 'full_name'] = 'Diego Castillo'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 681082, 'player_age'] = 25

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 669450, 'full_name'] = 'Bryson Stott'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 669450, 'player_age'] = 25

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 682928, 'full_name'] = 'Cooper Hummel'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 682928, 'player_age'] = 28

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 665922, 'full_name'] = 'C.J. Abrams'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 665922, 'player_age'] = 22

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 665650, 'full_name'] = 'Simon Muzziotti'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 665650, 'player_age'] = 24

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 645444, 'full_name'] = 'Lucius Fox'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 645444, 'player_age'] = 25

player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 660636, 'full_name'] = 'Jose Herrera'
player_age_lookup_merge_NL.loc[player_age_lookup_merge_NL['player_id'] == 660636, 'player_age'] = 25

player_age_lookup_merge_NL_final = player_age_lookup_merge_NL[['full_name','player_id','year','player_age']]
player_age_lookup_merge_NL_final['league'] = 'NL'
player_age_lookup_merge_NL_final

/var/folders/d8/89h35qn507zfzwn9__35p_bc0000gn/T/ipykernel_68308/3412345940.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_age_lookup_merge_NL_final['league'] = 'NL'


full_name  player_id  year  player_age league
0    Bryan De La Cruz     650559  2022        25.0     NL
1           Alec Bohm     664761  2022        25.0     NL
2            Ian Happ     664023  2022        27.0     NL
3       Austin Barnes     605131  2022        32.0     NL
4       Yonathan Daza     602074  2022        28.0     NL
..                ...        ...   ...         ...    ...
191    Luis Guillorme     641645  2022        27.0     NL
192     Aramis Garcia     605244  2022        29.0     NL
193        Lucius Fox     645444  2022        25.0     NL
194      Payton Henry     668721  2022        25.0     NL
195   Yonny Hernandez     660634  2022        24.0     NL

[196 rows x 5 columns]

In [81]:
player_age_lookup_merge_NL_final = player_age_lookup_merge_NL_final.reindex(columns = list_of_cols)
player_age_lookup_merge_NL_final

full_name  player_id  year  player_age league  Total_WAR  WAR_bat  \
0    Bryan De La Cruz     650559  2022        25.0     NL        NaN      NaN   
1           Alec Bohm     664761  2022        25.0     NL        NaN      NaN   
2            Ian Happ     664023  2022        27.0     NL        NaN      NaN   
3       Austin Barnes     605131  2022        32.0     NL        NaN      NaN   
4       Yonathan Daza     602074  2022        28.0     NL        NaN      NaN   
..                ...        ...   ...         ...    ...        ...      ...   
191    Luis Guillorme     641645  2022        27.0     NL        NaN      NaN   
192     Aramis Garcia     605244  2022        29.0     NL        NaN      NaN   
193        Lucius Fox     645444  2022        25.0     NL        NaN      NaN   
194      Payton Henry     668721  2022        25.0     NL        NaN      NaN   
195   Yonny Hernandez     660634  2022        24.0     NL        NaN      NaN   

     WAR_pit  b_ab  b_total_pa  ...  b_triple  b_home_run  b_strikeout  \
0        NaN   NaN         NaN  ...       NaN         NaN          NaN   
1        NaN   NaN         NaN  ...       NaN         NaN          NaN   
2        NaN   NaN         NaN  ...       NaN         NaN          NaN   
3        NaN   NaN         NaN  ...       NaN         NaN          NaN   
4        NaN   NaN         NaN  ...       NaN         NaN          NaN   
..       ...   ...         ...  ...       ...         ...          ...   
191      NaN   NaN         NaN  ...       NaN         NaN          NaN   
192      NaN   NaN         NaN  ...       NaN         NaN          NaN   
193      NaN   NaN         NaN  ...       NaN         NaN          NaN   
194      NaN   NaN         NaN  ...       NaN         NaN          NaN   
195      NaN   NaN         NaN  ...       NaN         NaN          NaN   

     b_walk  b_k_percent  b_bb_percent  batting_avg  slg_percent  \
0       NaN          NaN           NaN          NaN          NaN   
1       NaN          NaN           NaN          NaN          NaN   
2       NaN          NaN           NaN          NaN          NaN   
3       NaN          NaN           NaN          NaN          NaN   
4       NaN          NaN           NaN          NaN          NaN   
..      ...          ...           ...          ...          ...   
191     NaN          NaN           NaN          NaN          NaN   
192     NaN          NaN           NaN          NaN          NaN   
193     NaN          NaN           NaN          NaN          NaN   
194     NaN          NaN           NaN          NaN          NaN   
195     NaN          NaN           NaN          NaN          NaN   

     on_base_percent  on_base_plus_slg  
0                NaN               NaN  
1                NaN               NaN  
2                NaN               NaN  
3                NaN               NaN  
4                NaN               NaN  
..               ...               ...  
191              NaN               NaN  
192              NaN               NaN  
193              NaN               NaN  
194              NaN               NaN  
195              NaN               NaN  

[196 rows x 23 columns]

### Create Final 2022 database

In [85]:
frame_2022 = pd.concat([player_age_lookup_merge_AL_final,player_age_lookup_merge_NL_final])
frame_2022['player_age'] = frame_2022['player_age'].astype(int)

In [86]:
### Export CSV

frame_2022.to_csv(f'Complete/{new_year}_complete.csv')